In [1]:
import glob
import argparse
import datetime
import os
from os.path import basename, dirname, join
import random
import json

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

import data_generator
from data_generator import DataGeneratorFrom_npz_File



import matplotlib.pyplot as plt
from matplotlib import colors

from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Dense,
                                     Flatten,
                                     Dropout,
                                     BatchNormalization, 
                                     Conv2D, 
                                     MaxPooling2D)
from tensorflow.keras.regularizers import l2

random.seed(1)

# idir = '/tf/data/output/'
# input_dir_json = '/tf/data/input/'

idir = '/Data/preprocessing/output/'

input_dir_json = '/Data/'

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-vw8zg7q1 because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
train_ratio = 0.7
with open(f'{idir}processed_files.json') as fichier_json:
    all_nc = json.load(fichier_json)
npz_files=[]
for nc in all_nc :
    name = nc[:15]
    files = sorted(glob.glob(f'{idir}/output_keep_w/{name}/*.npz'))
    npz_files += files
random.shuffle(npz_files)
print(len(npz_files))
train_files_number = int(len(npz_files) * train_ratio)
train_files = npz_files[:train_files_number]
valid_files = npz_files[train_files_number:]

input_var_names = ['nersc_sar_primary', 'nersc_sar_secondary']
amsr2_var_names = [
    'btemp_6_9h',
    'btemp_6_9v',
    'btemp_7_3h',
    'btemp_7_3v',
    'btemp_10_7h',
    'btemp_10_7v',
    'btemp_18_7h',
    'btemp_18_7v',
    'btemp_23_8h',
    'btemp_23_8v',
    'btemp_36_5h',
    'btemp_36_5v',
    'btemp_89_0h',
    'btemp_89_0v']

# with open(f'{input_dir_json}vector_combinations.json') as fichier_json:
#     list_combi = json.load(fichier_json)['all_work_comb']

encoding ="hugo"
    
output_var_name = 'ice_type'

dims_input = np.load(npz_files[0])[input_var_names[0]].shape
dims_amsr2 = np.load(npz_files[0])[amsr2_var_names[0]].shape

params = {'dims_input':      (*dims_input, len(input_var_names)),
           'dims_amsr2':      (*dims_amsr2, len(amsr2_var_names)),
           'idir_json': input_dir_json,
           'output_var_name': output_var_name,
           'input_var_names': input_var_names,
           'amsr2_var_names': amsr2_var_names,
           'batch_size':      50,
           'encoding':        encoding,
           'shuffle_on_epoch_end': False,
           }

777697


In [3]:
training_generator = DataGeneratorFrom_npz_File(train_files, **params)


print(
    training_generator[0][0][0].shape, #sar
    training_generator[0][0][1].shape, #amsr2
    training_generator[0][1].shape, #output
)

# print('sar')
# print(training_generator[0][0][0][:1])
# print('amsr2')
# print(training_generator[0][0][1][:1])
# print('output')
# print(training_generator[0][1][:1])
# print('distance')
# print(training_generator[0][2][:1])

[91 40 95  4 30 93  4 30 91  3]
[91 20 87  6 80 83  5 -9 -9 -9]
[10 -9 93  6 -9 -9 -9 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[92 -9 91  8 -9 -9 -9 -9 -9 -9]
[91 80 91  7 20 87  6 -9 -9 -9]
[91 50 95  6 40 87  5 10 83  5]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[90 70 91  6 20 87  5 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[91 50 95  6 40 91  5 10 87  4]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[91 50 95  6 30 93  6 20 91  4]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[90 -9 91  6 -9 -9 -9 -9 -9 -9]
[91 40 95  5 40 93  4 20 91  3]
[91 80 91  7 20 87  6 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[91 20 87  6 80 83  5 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[91 60 97  7 30 96  6 10 93  6]
[10 -9 93  2 -9 -9 -9 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[ 2 -9 -

In [6]:
list_combi = ["0_0", "83_5", "93_6", "87_6", "95_4", "95_6", "91_5", "95_3", "95_5", "83_6", "93_4", "91_3", "83_4", "91_6", "93_7", "96_6", "91_4", "91_2", "93_5", "82_3", "87_5", "97_7", "87_3", "82_5", "82_4", "83_3", "93_2", "93_3", "87_4", "91_7", "95_7", "82_2"]
vector_param = training_generator[0][1]
print(vector_param[0:15])
# sar_sv = (np.delete(training_generator[0][0][0],1,axis=4)).squeeze()
# output = training_generator[0][1]
# print(sar_sv.squeeze().shape)
# print(output.shape)

[91 40 95  4 30 93  4 30 91  3]
[91 20 87  6 80 83  5 -9 -9 -9]
[10 -9 93  6 -9 -9 -9 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[92 -9 91  8 -9 -9 -9 -9 -9 -9]
[91 80 91  7 20 87  6 -9 -9 -9]
[91 50 95  6 40 87  5 10 83  5]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[90 70 91  6 20 87  5 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[91 50 95  6 40 91  5 10 87  4]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[91 50 95  6 30 93  6 20 91  4]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[90 -9 91  6 -9 -9 -9 -9 -9 -9]
[91 40 95  5 40 93  4 20 91  3]
[91 80 91  7 20 87  6 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[91 20 87  6 80 83  5 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[91 60 97  7 30 96  6 10 93  6]
[10 -9 93  2 -9 -9 -9 -9 -9 -9]
[ 2 -9 -9 -9 -9 -9 -9 -9 -9 -9]
[ 2 -9 -

In [5]:
test1=[0.10000000000000009, 0, 0.8999999999999999, 0]
test2=[0.19999999999999996, 0, 0.5, 0.3]


exemple =[92 -9 91  8 -9 -9 -9 -9 -9 -9]
